# Get_Resource_directory

<div class="alert alert-block alert-info">
<i>Hello there.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>get_resource_directory</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success">
    <h1> 1.</h1>
Below we are importing all the necessary modules from the ilorest library!.<br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li> redfish </li>
    <li>get_resource_directory</li>
</div>

In [1]:
import sys

from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

#from get_resource_directory import get_resource_directory

<div class = "alert alert-block alert-success">
    <h1> 2. </h1>
     A redfishobj parameter is required for the function get_resource_directory. The resource directory can be retrieved using this function from a Redfish API.<br>
    <i>Let me give you a walk through of it!</i>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.<br>

1. It tries to use the redfishobj object to access the resource directory URI. The server's resources are gathered in the resource directory.The server is not an HPE server and an error message is displayed to stderr if the resource directory URI cannot be discovered (KeyError is thrown). Then, the function returns null.
<br>
    
2. If the resource directory URI is located, the code uses the redfishobj.get method to send an <b>HTTP GET</b> request to retrieve the resource directory.
    <br>

3. A response status of 200 (OK) indicates a successful retrieval of the resource directory. The function assigns the list of resource instances to the resources variable and sends a message to stdout showing the location of the resource directory.
    <br>

4. If the response status isn't 200, the resource directory is either unavailable or missing. The resources variable is empty and an error is reported to stderr.
    
 <br>  
    
5. Finally the function get_resource_directory return the  <b>resources</b>.
    <br>

 </div>

In [1]:
def get_resource_directory(redfishobj):

    try:
        resource_uri = redfishobj.root.obj.Oem.Hpe.Links.ResourceDirectory['@odata.id']
    except KeyError:
        sys.stderr.write("Resource directory is only available on HPE servers.\n")
        return None

    response = redfishobj.get(resource_uri)
    resources = []

    if response.status == 200:
        sys.stdout.write("\tFound resource directory at /redfish/v1/resourcedirectory" + "\n\n")
        resources = response.dict["Instances"]
    else:
        sys.stderr.write("\tResource directory missing at /redfish/v1/resourcedirectory" + "\n")

    return resources


<div class = "alert alert-block alert-success">
    <h1> 3. </h1>This is get_gen function overview.<br>
    <i>Let me give you a walk through of it!</i>
    <br>


1. The _redfishobj object, which is presumed to be a Redfish response object, is used to retrieve the root response.
The server generation's default setting is set to 5 (iLO 5).<br>
<br>
    
2. By looking at the keys beneath the "Oem" key in the root response, it determines whether the OEM (Original Equipment Manufacturer) of the server is either "Hpe" or "Hp." Gencompany is set to True if one of these keys is discovered; otherwise, it is set to False..<br>
    <br>

3. If gencompany is True, the variable comp is changed to Hp otherwise, it stays None.
Comp is changed to Hpe if comp is not None and the <b>Oem</b> key contains the subkey Hpe in the root response.<br>
    <br>

4. The function then accesses nested dictionaries in the root response to see if the server's manager type is present.It fetches the manager type and firmware version if the manager type is present.
It thinks the server is an iLO 4 type on a Moonshot system if the manager type ends with CM (probably "Chassis Manager") in which case it sets ilogen to 4 and iloversion to None.<br>
    <br>
5. In the absence of that, it takes the management type's generation number and the firmware version number. These values are combined to create a decimal version number, which is then assigned to iloversion. Ilogen is given the extracted generation number.
<br>
    

6. A tuple including the server generation (ilogen) and version (iloversion) is the function's final output.

 </div>

In [ ]:
def get_gen(_redfishobj):
	rootresp = _redfishobj.root.obj
	#Default iLO 5
	ilogen = 5
	gencompany = next(iter(rootresp.get("Oem", {}).keys()), None) in ('Hpe', 'Hp')
	comp = 'Hp' if gencompany else None
	comp = 'Hpe' if rootresp.get("Oem", {}).get('Hpe', None) else comp
	if comp and next(iter(rootresp.get("Oem", {}).get(comp, {}).get("Manager", {}))).\
																get('ManagerType', None):
		ilogen = next(iter(rootresp.get("Oem", {}).get(comp, {}).get("Manager", {})))\
																		.get("ManagerType")
		ilover = next(iter(rootresp.get("Oem", {}).get(comp, {}).get("Manager", {}))).\
															  get("ManagerFirmwareVersion")
		if ilogen.split(' ')[-1] == "CM":
			# Assume iLO 4 types in Moonshot
			ilogen = 4
			iloversion = None
		else:
			ilogen = ilogen.split(' ')[1]
			iloversion = float(ilogen.split(' ')[-1] + '.' + \
											''.join(ilover.split('.')))
	return (ilogen, iloversion)


<div class="alert alert-block alert-warning">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class = "alert alert-block alert-success">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>